<a href="https://colab.research.google.com/github/nbashyal/debiasing_LLM/blob/main/WEAT_score_gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# load tokenizer and model
!pip install transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 12.7 MB/s eta 0:00:00


In [ ]:
# tokenize list of words, and calclate average embeddings 
import numpy as np
def get_average_embeddings(words):
  embeddings = []
  tokenizer.add_special_tokens({'pad_token': 'NONE'})
  for word in words:
    tokens = tokenizer.encode_plus(word, return_tensors= 'pt', padding= True, truncation=True)
    with torch.no_grad():
      embedding= model(**tokens)[0].mean(dim=1).squeeze()
    embeddings.append(embedding.numpy())
  return torch.tensor(np.mean(embeddings, axis=0))
  



In [ ]:
# calculate cosine similarity between 2 embeddings 
def cosine_sim(emb1, emb2):
  emb1= emb1/emb1.norm()
  emb2= emb2/emb2.norm()

  return torch.nn.functional.cosine_similarity(emb1.unsqueeze(0), emb2.unsqueeze(0))

In [ ]:
# Define the target sets for male and female 
male_words = ['he', 'him', 'his', 'man', 'boy', 'brother', 'father', 'uncle', 'grandfather']
female_words = ['she', 'her', 'hers', 'woman', 'girl', 'sister', 'mother', 'aunt', 'grandmother']

# Define the attribute sets (in this case occupations)
male_occupations = ['carpenter', 'electrician', 'plumber', 'mechanic', 'construction worker', 'firefighter', 'police officer', 'soldier', 'politics', 'mob']
female_occupations = ['nurse', 'teacher', 'librarian', 'receptionist', 'hairdresser', 'waitress', 'cleaner', 'babysitter', 'biologist']

# Compute the average embeddings for each set of words
average_male_embed = get_average_embeddings(male_words)
average_female_embed = get_average_embeddings(female_words)
average_male_occ_embed = get_average_embeddings(male_occupations)
average_female_occ_embed = get_average_embeddings(female_occupations)

# Compute the WEAT score
numerator = torch.dot(average_male_embed - average_female_embed, average_male_occ_embed - average_female_occ_embed)
denominator = torch.norm(average_male_embed - average_female_embed) * torch.norm(average_male_occ_embed - average_female_occ_embed)

WEAT = numerator / denominator
WEAT


tensor(0.0142)